<a href="https://colab.research.google.com/github/nisargnegi/Sherlocked_RAG_Llama3_LlamaParse_Groq/blob/main/Sherlocked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires protobuf<5,>=3.20, but you have protobuf 5.26.1 which is incompatible.
google-ai-generativelanguage 0.6.3 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-api-core 2.11.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0.dev0,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-aiplatform 1.51.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.26.1 which is incompatible.
google-cloud-bigquery-connection 1.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but y

In [3]:
import os
import textwrap
from pathlib import Path

from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")


def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))

In [6]:
import requests
from bs4 import BeautifulSoup
import os
from urllib.parse import urljoin
from PyPDF2 import PdfMerger

# URL of the webpage containing the PDF links
url = 'https://sherlock-holm.es/pdf/a4/1-sided/'

# Create directories to save the PDFs if they don't exist
os.makedirs('sherlock_pdfs', exist_ok=True)
os.makedirs('data', exist_ok=True)

# Get the webpage content
response = requests.get(url)
response.raise_for_status()  # Check that the request was successful

# Parse the webpage content
soup = BeautifulSoup(response.content, 'html.parser')

# Find all links to PDF files
pdf_links = soup.find_all('a', href=True)
pdf_urls = [link['href'] for link in pdf_links if link['href'].endswith('.pdf')]

# Initialize the PdfMerger
merger = PdfMerger()

# Define the size limit (500KB)
size_limit_kb = 500
size_limit_bytes = size_limit_kb * 1024

# Download each PDF and add to the merger
for pdf_url in pdf_urls:
    # Make the full URL if necessary
    full_pdf_url = urljoin(url, pdf_url)

    # Get the filename from the URL
    filename = pdf_url.split('/')[-1]

    # Download the PDF
    print(f'Downloading {filename}...')
    pdf_response = requests.get(full_pdf_url)
    pdf_response.raise_for_status()

    # Save the PDF to the directory
    pdf_path = os.path.join('sherlock_pdfs', filename)
    with open(pdf_path, 'wb') as f:
        f.write(pdf_response.content)

    # Check the size of the downloaded PDF
    file_size = os.path.getsize(pdf_path)
    if file_size > size_limit_bytes:
        print(f'{filename} is larger than 500KB and will be skipped.')
        os.remove(pdf_path)  # Remove the large file
    else:
        print(f'{filename} downloaded and added to the merger.')
        # Add the PDF to the merger
        merger.append(pdf_path)

# Write out the merged PDF
output_path = os.path.join('data', 'merged_sherlock.pdf')
merger.write(output_path)
merger.close()

print(f'All PDFs under 500KB have been merged into {output_path}.')


cano.pdf is larger than 500KB and will be skipped.
advs.pdf is larger than 500KB and will be skipped.
mems.pdf is larger than 500KB and will be skipped.
retn.pdf is larger than 500KB and will be skipped.
lstb.pdf is larger than 500KB and will be skipped.
case.pdf is larger than 500KB and will be skipped.
stud.pdf downloaded and added to the merger.
sign.pdf downloaded and added to the merger.
houn.pdf downloaded and added to the merger.
vall.pdf downloaded and added to the merger.
scan.pdf downloaded and added to the merger.
redh.pdf downloaded and added to the merger.
iden.pdf downloaded and added to the merger.
bosc.pdf downloaded and added to the merger.
five.pdf downloaded and added to the merger.
twis.pdf downloaded and added to the merger.
blue.pdf downloaded and added to the merger.
spec.pdf downloaded and added to the merger.
engr.pdf downloaded and added to the merger.
nobl.pdf downloaded and added to the merger.
bery.pdf downloaded and added to the merger.
copp.pdf downloaded

## Document Parsing

In [7]:
instruction = """The provided document is a Sherlock Holmes book.
This text contains details about the characters, plot, and setting of a Sherlock Holmes story.
It may include descriptions of crimes, investigations, and deductions.
Try to focus on factual information and avoid subjective analysis."""


parser = LlamaParse(
    api_key=userdata.get("LLAMA_PARSE"),
    result_type="markdown",
    parsing_instruction=instruction,
    max_timeout=5000,
)

llama_parse_documents = await parser.aload_data("./data/merged_sherlock.pdf",)

Started parsing the file under job_id 2a31452b-addd-4a9c-91b1-d2fbfb0bee45


In [8]:
parsed_doc = llama_parse_documents[0]

In [9]:
Markdown(parsed_doc.text[:4096])

A Study In Scarlet

Arthur Conan Doyle
---
This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the text or any medium it may be on, including but not limited to warranties of merchantability or fitness for a particular purpose. This text was formatted from various free ASCII and HTML variants. See http://sherlock-holm.es for an electronic form of this text and additional information about it. This text comes from the collection’s version 3.1.
---
# Table of contents

|Part I|Mr. Sherlock Holmes|5|
|---|---|---|
| |The Science Of Deduction|8|
| |The Lauriston Garden Mystery|12|
| |What John Rance Had To Tell|17|
| |Our Advertisement Brings A Visitor|20|
| |Tobias Gregson Shows What He Can Do|23|
| |Light In The Darkness|27|
|Part II|On The Great Alkali Plain|35|
| |The Flower Of Utah|39|
| |John Ferrier Talks With The Prophet|42|
| |A Flight For Life|44|
| |The Avenging Angels|48|
| |A Continuation Of The Reminiscences Of John Watson, M.D.|52|

The Conclusion

57
---


---
# PART I.

(Being a reprint from the reminiscences of John H. Watson, M.D., late of the Army Medical Department.)
---


---
# CHAPTER I.

# Mr. Sherlock Holmes

In the year 1878 I took my degree of Doctor of Medicine of the University of London, and proceeded to Netley to go through the course prescribed for surgeons in the army. Having completed my studies there, I was duly attached to the Fifth Northumberland Fusiliers as Assistant Surgeon. The regiment was stationed in India at the time, and before I could join it, the second Afghan war had broken out. On landing at Bombay, I learned that my corps had advanced through the passes, and was already deep in the enemy’s country. I followed, however, with many other officers who were in the same situation as myself, and succeeded in reaching Candahar in safety, where I found my regiment, and at once entered upon my new duties.

The campaign brought honours and promotion to many, but for me it had nothing but misfortune and disaster. I was removed from my brigade and attached to the Berkshires, with whom I served at the fatal battle of Maiwand. There I was struck on the shoulder by a Jezail bullet, which shattered the bone and grazed the subclavian artery. I should have fallen into the hands of the murderous Ghazis had it not been for the devotion and courage shown by Murray, my orderly, who threw me across a pack-horse, and succeeded in bringing me safely to the British lines.

Worn with pain, and weak from the prolonged hardships which I had undergone, I was removed, with a great train of wounded sufferers, to the base hospital at Peshawar. Here I rallied, and had already improved so far as to be able to walk about the wards, and even to bask a little upon the verandah, when I was struck down by enteric fever, that curse of our Indian possessions. For months my life was despaired of, and when at last I came to myself and became convalescent, I was so weak and emaciated that a medical board determined that not a day should be lost in sending me back to England. I was dispatched, accordingly, in the troopship Orontes, and landed a month later on Portsmouth jetty, with my health irretrievably ruined, but with permission from a paternal government to spend the next nine months in attempting to improve it.

I had neither kith nor kin in England, and was therefore as free as air—or as free as an income of eleven shillings and sixpence a day will permit a man to be. Under such circumstances, I naturally gravitated to London, that great cesspool into which all the loungers and idlers of the Empire are irresistibly drained. There I stayed for some time at a private hotel in the Strand, leading a comfortless, meaningless existence, and spending such money as I had, considerably more freely than I ought. So alarming did the state of my finances become, that I soon realized that I must either leave the metropolis and rusticate somewhere in the country, or that I must make a complete alteration in my style of living. Choosing the latter alte

In [10]:
document_path = Path("data/parsed_document.md")
with document_path.open("a") as f:
    f.write(parsed_doc.text)

## Vector Embeddings

In [11]:
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
len(docs)

2001

In [13]:
print(docs[0].page_content)

A Study In Scarlet

Arthur Conan Doyle

This text is provided to you “as-is” without any warranty. No warranties of any kind, expressed or implied, are made to you as to the text or any medium it may be on, including but not limited to warranties of merchantability or fitness for a particular purpose. This text was formatted from various free ASCII and HTML variants. See http://sherlock-holm.es for an electronic form of this text and additional information about it. This text comes from the collection’s version 3.1.

Table of contents

Part I Mr. Sherlock Holmes 5 The Science Of Deduction 8 The Lauriston Garden Mystery 12 What John Rance Had To Tell 17 Our Advertisement Brings A Visitor 20 Tobias Gregson Shows What He Can Do 23 Light In The Darkness 27 Part II On The Great Alkali Plain 35 The Flower Of Utah 39 John Ferrier Talks With The Prophet 42 A Flight For Life 44 The Avenging Angels 48 A Continuation Of The Reminiscences Of John Watson, M.D. 52

The Conclusion

57

PART I.

(Bein

In [14]:
embeddings = FastEmbedEmbeddings(model_name="BAAI/bge-base-en-v1.5")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/740 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/218M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

In [15]:
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # location=":memory:",
    path="./db",
    collection_name="document_embeddings",
)

In [16]:
%%time
query = "What is the most difficult case solved by Sherlock?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 260 ms, sys: 16 ms, total: 276 ms
Wall time: 283 ms


In [17]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("-" * 80)
    print()

text: it were only so." "What is it, Mr. Holmes? Have you a clue?" "An idea—an indication, no more. But the case certainly grows in interest. Unique, perfectly unique, and yet why not? I do not see any indications of bleeding on the line."

score: 0.4694499417018526
--------------------------------------------------------------------------------

text: The firemen had been much perturbed at the strange arrangements which they had found within, and still more so by discovering a newly severed human thumb upon a window-sill of the second floor. About sunset, however, their efforts were at last successful, 

score: 0.4589313304577645
--------------------------------------------------------------------------------

text: Honoured Colleague [it ran]: Since your esteemed visit I have thought much of your case, and though in your circumstances there are some special reasons for the treatment, I would none the less enjoin caution, as my results have shown that it is not withou

score: 0.455809

In [18]:
%%time
retriever = qdrant.as_retriever(search_kwargs={"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 278 ms, sys: 4 ms, total: 282 ms
Wall time: 283 ms


In [19]:
for doc in retrieved_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print("-" * 80)
    print()

id: 37af55d8d0ea4a1fb59d2d5170d5af1e

text: it were only so." "What is it, Mr. Holmes? Have you a clue?" "An idea—an indication, no more. But the case certainly grows in interest. Unique, perfectly unique, and yet why not? I do not see any indications of bleeding on the line."

--------------------------------------------------------------------------------

id: 922a2da711254c72b9d0cd004927561a

text: The firemen had been much perturbed at the strange arrangements which they had found within, and still more so by discovering a newly severed human thumb upon a window-sill of the second floor. About sunset, however, their efforts were at last successful, 

--------------------------------------------------------------------------------

id: e640d7b4cef94b478338fe33b52f0752

text: Honoured Colleague [it ran]: Since your esteemed visit I have thought much of your case, and though in your circumstances there are some special reasons for the treatment, I would none the less enjoin caution, a

## Reranking

In [20]:
compressor = FlashrankRerank(model="ms-marco-MiniLM-L-12-v2")
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:01<00:00, 19.1MiB/s]


In [21]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 2.69 s, sys: 191 ms, total: 2.88 s
Wall time: 2.82 s


3

In [22]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: d8deb588989d4b2c80335c7c15e4eaba

text: "The Bible!" I cried triumphantly.

"Good, Watson, good! But not, if I may say so, quite good enough! Even if I accepted the compliment for myself I could hardly name any volume which would be less likely to lie at the elbow of one of Moriarty’s associates

score: 2.5766439648577943e-05
--------------------------------------------------------------------------------

id: d37dc9b6337048a8908f4476108fbbe6

text: other ship. I spent three days in wiring to Dundee, "Wonderful!" cried Hopkins. "Wonderful!" and at the end of that time I had ascertained the names of the crew of the Sea Unicorn in 1883. When I found Patrick Cairns among the harpooners my research was ne

score: 1.662402064539492e-05
--------------------------------------------------------------------------------

id: 922a2da711254c72b9d0cd004927561a

text: The firemen had been much perturbed at the strange arrangements which they had found within, and still more so by discovering a n

## Q&A Over Document

In [23]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [24]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [25]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": True},
)

In [26]:
%%time
response = qa.invoke("What is the most difficult case solved by Sherlock?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Somewhere in the vaults of the bank of Cox and Co., at Charing Cross, there is a travel-worn and battered tin dispatch-box with my name, John H. Watson, M.D., Late Indian Army, painted upon the lid. It is crammed with papers, nearly all of which are records of cases to illustrate the curious problems which Mr. Sherlock Holmes had at various times to examine. Some, and not the least interesting, were complete failures, and as such will hardly bear narrating, since no final explanation is forthcoming. A problem without a solution may interest the student, but can hardly fail to annoy the casual reader. Among these unfinished tales is that of Mr. James Phillimore, who, stepping back into

In [27]:
print_response(response)

Based on the provided text, it is not possible to determine the most difficult case solved by
Sherlock Holmes. The text mentions several cases, including the case of Mr. James Phillimore, the
cutter Alicia, and Isadora Persano, but it does not provide a clear answer to the question.

However, it is worth noting that the text suggests that Sherlock Holmes has solved many cases, and
that he has a unique approach to solving mysteries. The conversation between Holmes, Gregson, and
Lestrade implies that Holmes has a reputation for being a skilled detective, and that he has a keen
mind for solving complex problems.

Additional helpful information:

* The text mentions that Sherlock Holmes has a collection of papers and records of his cases, which
are stored in a tin dispatch-box at the bank of Cox and Co.
* The text also mentions that Holmes has a habit of keeping his thoughts and theories to himself,
and that he often only reveals his conclusions when he has gathered all the necessary evide

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose": False},
)

In [38]:
%%time
response = qa.invoke("who was the great hound really?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: “A dog!” said Holmes. “By Jove, a curly-haired spaniel. Poor Mortimer will never see his pet again. Well, I do not know that this place contains any secret which we have not already fathomed. He could hide his hound, but he could not hush its voice, and hence came those cries which even in daylight were not pleasant to hear. On an emergency he could keep the hound in the out-house at Merripit, but it was always a risk, and it was only on the supreme day, which he regarded as the end of all his efforts, that he dared do it. This paste in the tin is no doubt the luminous mixture with which the creature was daubed. It was suggested, of course, by the story of the family hell-hound, and b

In [39]:
Markdown(response["result"])

**Answer:** The great hound was a curly-haired spaniel, not a supernatural creature.

**Additional helpful information:**

* The hound was a real dog, not a spectral or hellish creature, as suggested by the family legend.
* The dog was used by Stapleton to create the illusion of a supernatural hound, in an attempt to frighten Sir Charles to death.
* The dog was kept in the out-house at Merripit House, and was occasionally taken out to create the illusion of a ghostly hound on the moor.
* The dog was daubed with a luminous mixture to make it appear eerie and supernatural.

In [40]:
%%time
response = qa.invoke("Address of Sherlock Holmes?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: ‘Where could I find him?’

‘Oh, at his new offices. He did tell me the address. Yes, 17 King Edward Street, near St. Paul’s.’

“I started off, Mr. Holmes, but when I got to that address it was a manufactory of artificial knee-caps, and no one in it had ever heard of either Mr. William Morris or Mr. Duncan Ross.”

“And what did you do then?” asked Holmes.

“I went home to Saxe-Coburg Square, and I took the advice of my assistant. But he could not help me in any way. He could only say that if I waited I should hear by post. But that was not quite good enough, Mr. Holmes. I did not wish to lose such a place without a struggle, so, as I had heard that you were good enough to give advice t

In [41]:
print_response(response)

The address of Sherlock Holmes is not explicitly mentioned in the provided text. However, it is
implied that his office is located near Saxe-Coburg Square, as Mr. Jabez Wilson mentions going home
to Saxe-Coburg Square before visiting Sherlock Holmes.


In [42]:
%%time
response = qa.invoke(
    "How did Irene trick Sherlock?"
)

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: A professional case of great gravity was engaging my own attention at the time, and the whole of next day I was busy at the bedside of the sufferer. It was not until close upon six o’clock that I found myself free and was able to spring into a hansom and drive to Baker Street, half afraid that I might be too late to assist at the denouement of the little mystery. I found Sherlock Holmes alone, however, half asleep, with his long, thin form curled up in the recesses of his armchair. A formidable array of bottles and test-tubes, with the pungent cleanly smell of hydrochloric acid, told me that he had

been called upon to fathom. Once only had I known him to fail, in the case of the King

In [43]:
print_response(response)

**Answer:** Irene Adler, also known as "the woman," tricked Sherlock Holmes by outsmarting him and
escaping his detection in the case of the King of Bohemia.

**Additional information:** Irene Adler was a clever and cunning opponent who managed to outwit
Sherlock Holmes, which was a rare occurrence. Her cleverness and intelligence earned her a special
place in Holmes' memory, and he referred to her as "the woman" in later conversations.


In [44]:
%%time
response = qa.invoke(
    "How does Sherlock feel about his family?"
)

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: During my long and intimate acquaintance with Mr. Sherlock Holmes I had never heard him refer to his relations, and hardly ever to his own early life. This reticence upon his part had increased the somewhat inhuman effect which he produced upon me, until sometimes I found myself regarding him as an isolated phenomenon, a brain without a heart, as deficient in human sympathy as he was pre-eminent in intelligence. His aversion to women and his disinclination to form new friendships were both typical of his unemotional character, but not more so than his complete suppression of every reference to his own people. I had come to believe that he was an orphan with no relatives living, but on

In [45]:
print_response(response)

**Answer:** Sherlock Holmes is reticent about his family and rarely speaks about them. However, he
does mention his brother Mycroft, indicating that he has a sibling.

**Additional information:**

* Sherlock's ancestors were country squires, and his grandmother was the sister of Vernet, a French
artist.
* He believes that his faculty of observation and deduction may be hereditary, citing his brother
Mycroft as an example of someone who possesses these traits to a greater degree than he does.


In [46]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
!zip -r my_directory.zip db

  adding: db/ (stored 0%)
  adding: db/meta.json (deflated 55%)
  adding: db/collection/ (stored 0%)
  adding: db/collection/document_embeddings/ (stored 0%)
  adding: db/collection/document_embeddings/storage.sqlite (deflated 52%)
  adding: db/.lock (stored 0%)
